In [1]:
import json
import pandas as pd
import sqlite3
import sys

input_file = r"C:\TakW\meta_Clothing_Shoes_and_Jewelry.jsonl" # each line in the file represents a separate JSON object

chunk_size_meta = 500000  # Number of items to read
chunk_meta = []

# Load JSONL file line by line Using chunk based load to save up Memory
with open(input_file, 'r') as file:
    for i, line in enumerate(file):
        chunk_meta.append(json.loads(line.strip()))  # Parse each line as JSON
        if len(chunk_meta) == chunk_size_meta:  # Stop after reading the specified number of items
            break

# size check
print(f"Loaded {len(chunk_meta)} items.")
print(type(chunk_meta) )

df_meta = pd.DataFrame(chunk_meta)
df_meta.drop(columns=["images"],inplace=True) # remove unused array strc. column


# Expland categories column
df_meta['categories'] = df_meta['categories'].apply(lambda x: x if type(x) is list else [])
categories_expanded = pd.DataFrame(df_meta['categories'].tolist(), index=df_meta.index)

list1=[]

for i in range(categories_expanded.shape[1]):
    list1.append("categories_"+str(i))
categories_expanded.columns=list1


df_meta_expand = pd.concat([df_meta[["parent_asin","main_category","title","price","categories","store"]], categories_expanded], axis=1)\
.drop(columns=['categories'])

#-----------------------------------------------

# Calculate Brand column
pd.set_option('display.max_colwidth', None)
chunk_size = 10000  # Adjust the chunk size calling Json normalize with chunk both memory saving and push for vectorize operation escape iterations.
chunks = []

for start in range(0, len(df_meta), chunk_size):
    chunk = df_meta[["details"]].iloc[start:start + chunk_size]
    details_flattened_chunk = pd.json_normalize(chunk['details'])
    chunk['Brand'] = details_flattened_chunk.get('Brand')
    chunks.append(chunk)

df_processed = pd.concat(chunks, ignore_index=True)
df_meta_expand["Brand"]=df_processed["Brand"]


# add root_category column

category_columns = ["categories_"+str(i) for i in range(7, -1, -1)]
df_meta_expand["root_category"]=df_meta_expand[category_columns].loc[:, "categories_7":"categories_0"].bfill(axis=1).iloc[:, 0]

#-----------------------------------------------

# Create an SQLite database in memory 
conn = sqlite3.connect(':memory:')  # Use ':memory:' for an in-memory database
cursor = conn.cursor()

# CREATE TABLE PRODUCT DIMENSION

df_product_dm=df_meta_expand[["parent_asin","main_category","title","price","Brand","store","root_category"]]

create_table_sql = """
CREATE TABLE IF NOT EXISTS product_dimension (
    parent_asin TEXT PRIMARY KEY,
    main_category TEXT ,
    title TEXT ,
    price REAL,
    brand TEXT,
    store_name TEXT,
    root_category TEXT 
);
"""
# Execute the CREATE TABLE statement
cursor.execute("DROP TABLE IF EXISTS product_dimension")  
cursor.execute(create_table_sql)
conn.commit()

# Save raw data to SQLite
df_product_dm.to_sql("raw_product", conn, if_exists="replace", index=False)


insert_product = """
INSERT OR IGNORE INTO product_dimension
SELECT *
FROM raw_product;
"""

cursor.execute(insert_product)
conn.commit()

#-----------------------------------------------


# Create STORE_DIMENSION table

# Save raw data to SQLite
df_meta[["store"]].to_sql("raw_store", conn, if_exists="replace", index=False)


create_store_table= """CREATE TABLE IF NOT EXISTS store_dimension (
    store_id INTEGER PRIMARY KEY AUTOINCREMENT,
    store_name VARCHAR(255) NOT NULL
); """

# Insert unique store names into the STORE_DIMENSION table
insert_store="""INSERT OR IGNORE INTO store_dimension (STORE_NAME)
SELECT DISTINCT store
FROM raw_store WHERE store IS NOT NULL ;"""



#cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS store_dimension")
conn.commit()


cursor.execute(create_store_table)
conn.commit()

cursor.execute(insert_store)
conn.commit()

#query_result = pd.read_sql_query("SELECT * FROM STORE_DIMENSION", conn)



#-----------------------------------------------

# FINAL FORMAT FOR CATEGORY DIMENSION

df_category = df_meta_expand[["categories_0","categories_1","categories_2","categories_3","categories_4","categories_5","categories_6","categories_7"]]\
.drop_duplicates()


# Save raw data to SQLite
df_category.to_sql("raw_categories", conn, if_exists="replace", index=False)

# Create a hierarchical category table
create_hierarchy_table = """
CREATE TABLE IF NOT EXISTS hierarchical_category (
    category_id INTEGER PRIMARY KEY AUTOINCREMENT,
    category_name TEXT UNIQUE NOT NULL,
    parent_category_id INTEGER
);
"""

# Insert root categories (categories_0) into the hierarchical_category table
insert_root_categories = """
INSERT OR IGNORE INTO hierarchical_category (category_name, parent_category_id)
SELECT DISTINCT categories_0 AS category_name, NULL AS parent_category_id
FROM raw_categories;
"""

# Insert subcategories (categories_1) linked to their parent (categories_0)
insert_subcategories_1 = """
INSERT OR IGNORE INTO hierarchical_category (category_name, parent_category_id)
SELECT DISTINCT rc.categories_1 AS category_name, hc.category_id AS parent_category_id
FROM raw_categories rc
JOIN hierarchical_category hc ON rc.categories_0 = hc.category_name
WHERE rc.categories_1 IS NOT NULL;
"""

# Insert subcategories (categories_2) linked to their parent (categories_1)
insert_subcategories_2 = """
INSERT OR IGNORE INTO hierarchical_category (category_name, parent_category_id)
SELECT DISTINCT rc.categories_2 AS category_name, hc.category_id AS parent_category_id
FROM raw_categories rc
JOIN hierarchical_category hc ON rc.categories_1 = hc.category_name
WHERE rc.categories_2 IS NOT NULL;
"""

# Insert subcategories (categories_3) linked to their parent (categories_2)
insert_subcategories_3 = """
INSERT OR IGNORE INTO hierarchical_category (category_name, parent_category_id)
SELECT DISTINCT rc.categories_3 AS category_name, hc.category_id AS parent_category_id
FROM raw_categories rc
JOIN hierarchical_category hc ON rc.categories_2 = hc.category_name
WHERE rc.categories_3 IS NOT NULL;
"""

# Insert subcategories (categories_4) linked to their parent (categories_3)
insert_subcategories_4 = """
INSERT OR IGNORE INTO hierarchical_category (category_name, parent_category_id)
SELECT DISTINCT rc.categories_4 AS category_name, hc.category_id AS parent_category_id
FROM raw_categories rc
JOIN hierarchical_category hc ON rc.categories_3 = hc.category_name
WHERE rc.categories_4 IS NOT NULL;
"""

# Insert subcategories (categories_5) linked to their parent (categories_4)
insert_subcategories_5 = """
INSERT OR IGNORE INTO hierarchical_category (category_name, parent_category_id)
SELECT DISTINCT rc.categories_5 AS category_name, hc.category_id AS parent_category_id
FROM raw_categories rc
JOIN hierarchical_category hc ON rc.categories_4 = hc.category_name
WHERE rc.categories_5 IS NOT NULL;
"""

# Insert subcategories (categories_6) linked to their parent (categories_5)
insert_subcategories_6 = """
INSERT OR IGNORE INTO hierarchical_category (category_name, parent_category_id)
SELECT DISTINCT rc.categories_6 AS category_name, hc.category_id AS parent_category_id
FROM raw_categories rc
JOIN hierarchical_category hc ON rc.categories_5 = hc.category_name
WHERE rc.categories_6 IS NOT NULL;
"""

# Insert subcategories (categories_7) linked to their parent (categories_6)
insert_subcategories_7 = """
INSERT OR IGNORE INTO hierarchical_category (category_name, parent_category_id)
SELECT DISTINCT rc.categories_7 AS category_name, hc.category_id AS parent_category_id
FROM raw_categories rc
JOIN hierarchical_category hc ON rc.categories_6 = hc.category_name
WHERE rc.categories_7 IS NOT NULL;
"""



# Execute SQL
#cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS hierarchical_category")
conn.commit()

cursor.execute(create_hierarchy_table)
conn.commit()

cursor.execute(insert_root_categories)
conn.commit()

cursor.execute(insert_subcategories_1)
conn.commit()

cursor.execute(insert_subcategories_2)
conn.commit()

cursor.execute(insert_subcategories_3)
conn.commit()

cursor.execute(insert_subcategories_4)
conn.commit()

cursor.execute(insert_subcategories_5)
conn.commit()

cursor.execute(insert_subcategories_6)
conn.commit()

cursor.execute(insert_subcategories_7)
conn.commit()


#query_result = pd.read_sql_query("SELECT * FROM hierarchical_category", conn)


#-----------------------------------------------



# DataFrame to the SQLite table
df_product_dm[["parent_asin"]].to_sql('Item_Meta_Check', conn, index=False, if_exists='replace')


# DQ Check for Unique
query = "SELECT parent_asin FROM Item_Meta_Check group by parent_asin having count(1)>1"
result_df_meta = pd.read_sql_query(query, conn)
if ( result_df_meta.shape[0] >0 ):
    print("Duplicate key found !!")
    sys.exit(1)
else:
    print("No Duplicate key found !!")
    
# DQ Null value check ( main_category  )
Null_check=df_product_dm['main_category'].isnull().sum() 
print("Null check Main Category:")
print(Null_check)

# DQ Invalid_Price check ( price  )
df_product_dm['price'] = pd.to_numeric(df_product_dm['price'], errors='coerce')
invalid_prices = df_product_dm[df_product_dm['price'] <= 0] ["parent_asin"]
print("invalid_prices:")
print(invalid_prices)
#-----------------------------------------------



# Display the results
#print(result_df_meta)
#print(df_meta.columns)

#import sys
#sys.exit(1) # stop execution
print("DONE")

Loaded 500000 items.
<class 'list'>
No Duplicate key found !!
Null check Main Category:
8041
invalid_prices:
Series([], Name: parent_asin, dtype: object)
DONE


C:\Users\OZGUR\AppData\Local\Temp\ipykernel_15812\4273756434.py:290: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_product_dm['price'] = pd.to_numeric(df_product_dm['price'], errors='coerce')


In [2]:
import json

input_file = r"C:\TakW\Clothing_Shoes_and_Jewelry.jsonl" # each line in the file represents a separate JSON object

chunk_review_size = 500000  # Number of items to read Using chunk based load to save up Memory
chunk_review = []

# Process JSONL file line by line
with open(input_file, 'r') as file:
    for i, line in enumerate(file):
        chunk_review.append(json.loads(line.strip()))  # Parse each line as JSON
        if len(chunk_review) == chunk_review_size:  # Stop after reading the specified number of items
            break

# Do something with the first chunk
print(f"Loaded {len(chunk_review)} items.")
print(type(chunk_review) )

df_rv = pd.DataFrame(chunk_review)
df_rv.timestamp = pd.to_datetime(df_rv.timestamp, unit='ms')
df_rv["timestamp_month"] = df_rv.timestamp.dt.month
df_rv.drop(columns=["images"],inplace=True) # remove unused array strc. column

# DQ Invalid Rating check ( Rating  )
df_rv['rating'] = pd.to_numeric(df_rv['rating'], errors='coerce')
invalid_rating = df_rv[ (df_rv['rating'] < 0) |  (df_rv['rating'] >5 )  ] [["asin","parent_asin"]]
print("invalid_rating:")
print(invalid_rating)
#


# CREATE FACT REVIEW TABLE

create_reviews_table_sql = """
CREATE TABLE IF NOT EXISTS fact_reviews (
    rating REAL NOT NULL,
    title TEXT NOT NULL,
    text TEXT NOT NULL,
    asin TEXT NOT NULL,
    parent_asin TEXT NOT NULL,
    user_id TEXT NOT NULL,
    timestamp TEXT NOT NULL,
    helpful_vote INTEGER,
    verified_purchase BOOLEAN,
    timestamp_month INTEGER NOT NULL
);
"""
# Execute the CREATE TABLE statement
cursor.execute("DROP TABLE IF EXISTS fact_reviews")  # Ensure a fresh start
cursor.execute(create_reviews_table_sql)
conn.commit()

# Save raw data to SQLite
df_rv.to_sql("raw_review", conn, if_exists="replace", index=False)

insert_product = """
INSERT OR IGNORE INTO fact_reviews
SELECT *
FROM raw_review;
"""

cursor.execute(insert_product)
conn.commit()


print("DONE")
# Display the results
#print(result_df_rv)
#print(df_rv.columns)

Loaded 500000 items.
<class 'list'>
invalid_rating:
Empty DataFrame
Columns: [asin, parent_asin]
Index: []
DONE


In [3]:
# DataFrame to the SQLite table
df_meta_expand[["parent_asin","main_category","categories_1"]].to_sql('result_df_meta_2', conn, index=False, if_exists='replace')
df_rv[["parent_asin","rating","timestamp_month"]].to_sql('Review_subset_1', conn, index=False, if_exists='replace')


# ● Average review rating per category per month. (using main_category)
query = "SELECT \
    META.main_category,REV.timestamp_month, IFNULL( AVG(REV.rating) ,0) AS AVG_RATING FROM  \
    product_dimension META  LEFT JOIN fact_reviews REV ON \
    META.parent_asin = REV.parent_asin where REV.timestamp_month IS NOT NULL AND META.main_category IS NOT NULL\
    GROUP BY  META.main_category, REV.timestamp_month"


result_df_meta_rv = pd.read_sql_query(query, conn)

display( result_df_meta_rv )


# ● Average review rating per category per month. (using root_category)
query = "SELECT \
    META.root_category,REV.timestamp_month, IFNULL( AVG(REV.rating) ,0) AS AVG_RATING FROM  \
    product_dimension META  LEFT JOIN fact_reviews REV ON \
    META.parent_asin = REV.parent_asin where REV.timestamp_month IS NOT NULL AND META.root_category IS NOT NULL\
    GROUP BY  META.root_category, REV.timestamp_month"


result_df_meta_rv_2 = pd.read_sql_query(query, conn)

display( result_df_meta_rv_2 )
print("DONE")

,main_category,timestamp_month,AVG_RATING
0,AMAZON FASHION,1,4.342851
1,AMAZON FASHION,2,4.306569
2,AMAZON FASHION,3,4.307585
3,AMAZON FASHION,4,4.285355
4,AMAZON FASHION,5,4.268498
...,...,...,...
198,Toys & Games,9,3.666667
199,Toys & Games,10,4.238095
200,Toys & Games,11,4.257143
201,Toys & Games,12,4.371429


,root_category,timestamp_month,AVG_RATING
0,100% Plus & Featured Brands,1,3.0
1,100% Plus (No Title Match),2,5.0
2,100% Plus (No Title Match),11,1.0
3,4th of July Up to 60% Off,2,5.0
4,4th of July Up to 60% Off,3,4.0
...,...,...,...
6625,Yoga,8,5.0
6626,Yoga,10,5.0
6627,Yoga,11,5.0
6628,Yoga,12,4.0


DONE


In [4]:
#● Analysis of review rating per brand per month.

query = "SELECT \
    META.Brand,REV.timestamp_month, \
    IFNULL( COUNT(REV.rating) ,0) AS REV_COUNT ,\
    IFNULL( AVG(REV.rating) ,0) AS AVG_RATING \
    FROM  \
    product_dimension META  LEFT JOIN fact_reviews REV ON \
    META.parent_asin = REV.parent_asin \
    where REV.timestamp_month IS NOT NULL and META.Brand IS NOT NULL \
    GROUP BY  META.Brand, REV.timestamp_month ORDER BY avg_rating DESC"
result_df_meta_rv_2 = pd.read_sql_query(query, conn)
display( result_df_meta_rv_2 )



# Top 5 Rated brands
top_5 = result_df_meta_rv_2.nlargest(5, 'AVG_RATING')

# Bottom 5 Rated brands
bottom_5 = result_df_meta_rv_2.nsmallest(5, 'AVG_RATING')

print("Top 5 Rated Brands:")
print(top_5)

print("\nBottom 5 Rated Brands:")
print(bottom_5)


# FINDING OUTLIERS for REV_COUNT


# Calculate IQR
Q1 = result_df_meta_rv_2['REV_COUNT'].quantile(0.25)
Q3 = result_df_meta_rv_2['REV_COUNT'].quantile(0.75)
IQR = Q3 - Q1

# Define outlier bounds
Low_Value = Q1 - 1.5 * IQR
High_Value = Q3 + 1.5 * IQR

# Find outliers
Outliers = result_df_meta_rv_2[ (result_df_meta_rv_2['REV_COUNT'] < Low_Value) | (result_df_meta_rv_2['REV_COUNT'] > High_Value)  ] 
print("Outliers:")
print(Outliers[["brand","timestamp_month","REV_COUNT"]])


,brand,timestamp_month,REV_COUNT,AVG_RATING
0,ANOGOL,4,1,5.0
1,ANOGOL,11,1,5.0
2,ARIAT,2,1,5.0
3,ARIAT,3,1,5.0
4,ARIAT,5,1,5.0
...,...,...,...,...
123,OZERO,10,1,2.0
124,F&L CLASSIC,4,1,1.0
125,FALAN MULE,8,1,1.0
126,HerBose,6,1,1.0


Top 5 Rated Brands:
    brand  timestamp_month  REV_COUNT  AVG_RATING
0  ANOGOL                4          1         5.0
1  ANOGOL               11          1         5.0
2   ARIAT                2          1         5.0
3   ARIAT                3          1         5.0
4   ARIAT                5          1         5.0

Bottom 5 Rated Brands:
           brand  timestamp_month  REV_COUNT  AVG_RATING
124  F&L CLASSIC                4          1         1.0
125   FALAN MULE                8          1         1.0
126      HerBose                6          1         1.0
127     Quickwig                6          1         1.0
122         Boao                1          1         2.0
Outliers:
       brand  timestamp_month  REV_COUNT
82   ESTALON               12          4
83     Sanuk                8          6
84   Ldurian               11         15
85     OZERO               12         10
86     Sanuk                4          5
90   ESTALON                3          4
91   ESTALON     

In [5]:
query_result = pd.read_sql_query("SELECT * FROM store_dimension", conn)
display(query_result)

query_result = pd.read_sql_query("SELECT * FROM hierarchical_category", conn)
display(query_result)


query_result = pd.read_sql_query("SELECT * FROM product_dimension", conn)
display(query_result)


query_result = pd.read_sql_query("SELECT * FROM fact_reviews", conn)
display(query_result)

,store_id,store_name
0,1,BALEAF
1,2,Merrell
2,3,SAS
3,4,SheIn
4,5,Michael Kors
...,...,...
75155,75156,Peony.T
75156,75157,RisonDesign
75157,75158,Riverchan
75158,75159,LLQQ


,category_id,category_name,parent_category_id
0,1,"Clothing, Shoes & Jewelry",NaN
1,2,"Shoe, Jewelry & Watch Accessories",NaN
2,3,Women,1.0
3,4,Novelty & More,1.0
4,5,Luggage & Travel Gear,1.0
...,...,...,...
1421,2056,Breast Lift Tape,663.0
1422,2060,Camisoles,1890.0
1423,2065,Beanies & Knit Hats,864.0
1424,2071,Headwraps,864.0


,parent_asin,main_category,title,price,brand,store_name,root_category
0,B09X1MRDN6,AMAZON FASHION,BALEAF Women's Long Sleeve Zip Beach Coverup UPF 50+ Sun Protection Hooded Cover Up Shirt Dress with Pockets,31.99,None,BALEAF,Cover-Ups
1,B073C4Q7W8,AMAZON FASHION,Merrell Work Moab 2 Vent Waterproof SR Boulder,None,None,Merrell,Hiking Shoes
2,B0944VG4Y4,AMAZON FASHION,"SAS Women's, Relaxed Sandal",188.95,None,SAS,Flats
3,B08JGGF5TJ,AMAZON FASHION,SheIn Women's Basic Stretch Plaid Mini Bodycon A-Line Pencil Skirt,12.99,None,SheIn,Skirts
4,B00ZQMM6BI,AMAZON FASHION,"Michael Kors Cindy, Women’s Cross-Body Bag",None,None,Michael Kors,Crossbody Bags
...,...,...,...,...,...,...,...
499995,B09M8Y5KGJ,AMAZON FASHION,"Humble Chic Small Hoop Earrings for Women - Hypoallergenic Lightweight Wire Threader Loop Drop Dangles, Safe for Sensitive Ears, 925 White - 0.5 inch",None,None,Humble Chic,Hoop
499996,B083LZ823P,All Beauty,Quinn Cosplay Costume Accessory Suspender Belt Necklacek Earring Rings Tattoo Stickers,None,None,LILLIWEEN,Men's Halloween Costumes
499997,B00AR5G228,AMAZON FASHION,Sterling Silver Diamond Cut Ring - Sterling Silver Geometric Ring,None,None,StarShine Jewelry,Statement
499998,B00332FHV4,AMAZON FASHION,"Amazon Collection Sterling Silver Claddagh Celtic Knot Pendant Necklace with Rolo Chain, 18""",None,None,Amazon Collection,Pendant Necklaces


,rating,title,text,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,timestamp_month
0,3.0,Arrived Damaged : liquid in hub locker!,"Unfortunately Amazon in their wisdom (cough, cough) decided to ship the snowsuit in a vinyl bag with holes in it! There was no other bag to protect the snowsuit inside vinyl bag with all the holes. This is what happened: Arrived in hub locker. It was the very top locker. Opened it & pulled the pkg out getting a very wet & nasty surprise at the same time. My senses were assaulted. Smells like tea tree oil. Feels like conditioner or lotion. I can’t understand how the delivery person a) didn’t smell that mess when they shoved the pkg in b) didn’t see the mess when they shoved it in - tho if they were short I guess that would explain it bc I’m 5’10” & I didn’t see it until the pkg was in my hands. The locker was up high & dark, but I could smell it the minute I walked into the hub locker room. I happen to be extremely allergic to tea tree oil. It’s made from mellaleuca trees which grow all over southwest Florida where I was raised. Tho native to Australia they were used to help drain the swamps of Florida generations ago bc they soak up so much water. It has a very distinct smell & unfortunately is frequently used as a “carrier oil” for other oils & body lotions, shampoos, conditioners, etc., so I have to be hyper-vigilant to avoid exposure. Instead- Last night I got to suck down Benadryl & pray my throat didn’t close while keeping my epi-pens handy. The liquid went through the holes & onto the snowsuit as you can clearly see in the pictures. So I guess the suit is waterproof. Looks true to size. Elastic appears stretchy. I’m super annoyed that this arrived in this condition & could have easily been avoided by amazon properly wrapping it to avoid the elements whatever they may be.",B096S6LZV4,B09NSZ5QMF,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2023-03-04 14:06:07.351000,0,1,3
1,3.0,Useless under 40 degrees.,"Useless under 40 degrees unless you’re just running to the mailbox & back & don’t mind freezing during that jog. Zero interior pockets so nowhere to put glasses, iPhone, etc other than the 2 exterior pockets which should be larger imo as they are the only pockets. Seems to me like they used a too large needle with the thread in mine bc it’s like the air comes through every single hole the thread is in that they used for decoration/insulation to give it a quilted effect. With cotton fabrics you wash it & the material will shrink around the thread & you don’t have the air coming through the too large holes made by the needle. I usually stick with the non-quilted versions of non-cotton coats fit this reason. It looks nice, but it does not keep me warm in northern Colorado where we dip into negative temps & have crazy high winds. I was not impressed with it. Got it in tye try before you buy program without realizing I would have to pay for bags to return the items I tried or that I would have to drop them off at ups. I’m disabled. I don’t drive. So fun to pay for taxis to return stuff when Amazon has a flipping hub at my apt complex. Why can’t they make returns simpler? Delivered to the hub & I should be able to return it to the hub imo. Really ticks me off I don’t get anything delivered to my door anymore since moving to an apt. As a disabled person it’s a massive pain in my tush. Every step is agony & the stuff weighs more than I am supposed to lift. It’s a long walk when you’re in chronic pain. Should have sent it back with the rest bc now I’ll have to pay for another $3 bag & taxis, but I wanted to try it in colder weather since it matched the color of my boots. My bad. Useless in cold weather. The other issue I noticed was I tried 3 Columbia jackets in the same size (xl) bc I like to wear fleece & layers under my coats due to the temps here & the difference in sizing was ridiculous. This was a true xl while the other two were not. Easily managed camisole, T-shirt, & long sleeved, very thick, 

In [6]:
"""
query_result = pd.read_sql_query("SELECT * FROM store_dimension", conn)
#display(query_result)

query_result=pd.read_sql_query("SELECT STORE_NAME, COUNT(*)  FROM store_dimension GROUP BY STORE_NAME HAVING COUNT(*) > 1", conn);
#display(query_result)

query_result = pd.read_sql_query("SELECT * FROM hierarchical_category", conn)
#display(query_result)


query_result = pd.read_sql_query("SELECT count(1) FROM product_dimension", conn)
display(query_result)


query_result = pd.read_sql_query("SELECT distinct root_category FROM product_dimension", conn)
display(query_result)

query_result = pd.read_sql_query("SELECT parent_asin FROM product_dimension group by parent_asin having count(1)>1", conn)
query_result.head(10)

query_result = pd.read_sql_query("SELECT count(1) FROM fact_reviews", conn)
display(query_result)

"""

'\nquery_result = pd.read_sql_query("SELECT * FROM store_dimension", conn)\n#display(query_result)\n\nquery_result=pd.read_sql_query("SELECT STORE_NAME, COUNT(*)  FROM store_dimension GROUP BY STORE_NAME HAVING COUNT(*) > 1", conn);\n#display(query_result)\n\nquery_result = pd.read_sql_query("SELECT * FROM hierarchical_category", conn)\n#display(query_result)\n\n\nquery_result = pd.read_sql_query("SELECT count(1) FROM product_dimension", conn)\ndisplay(query_result)\n\n\nquery_result = pd.read_sql_query("SELECT distinct root_category FROM product_dimension", conn)\ndisplay(query_result)\n\nquery_result = pd.read_sql_query("SELECT parent_asin FROM product_dimension group by parent_asin having count(1)>1", conn)\nquery_result.head(10)\n\nquery_result = pd.read_sql_query("SELECT count(1) FROM fact_reviews", conn)\ndisplay(query_result)\n\n'

In [7]:
query_result = pd.read_sql_query("SELECT * FROM hierarchical_category", conn)
display(query_result)


,category_id,category_name,parent_category_id
0,1,"Clothing, Shoes & Jewelry",NaN
1,2,"Shoe, Jewelry & Watch Accessories",NaN
2,3,Women,1.0
3,4,Novelty & More,1.0
4,5,Luggage & Travel Gear,1.0
...,...,...,...
1421,2056,Breast Lift Tape,663.0
1422,2060,Camisoles,1890.0
1423,2065,Beanies & Knit Hats,864.0
1424,2071,Headwraps,864.0


In [10]:
details_flattened_chunk

,Department,Date First Available,Package Dimensions,Item model number,Manufacturer,Is Discontinued By Manufacturer,Color,Brand,Material,Style,...,Best Sellers Rank.Tactical Bags & Packs,Best Sellers Rank.Men's Soccer Clothing,Best Sellers Rank.Women's Ear Cuffs & Wraps,Lens Coating Description,Best Sellers Rank.Men's Polo Shirts,Best Sellers Rank.Men's Golf Shirts,Best Sellers Rank.Men's Activewear,Best Sellers Rank.Boys' Skiing & Snowboarding Gloves,Best Sellers Rank.Women's Rain Footwear,Best Sellers Rank.Women's Mid-Calf Boots
0,Womens,"July 7, 2022",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,girls,"August 3, 2012",12 x 8 x 4 inches; 2 Pounds,Bianca - K,pediped,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,womens,"December 2, 2021",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,womens,"October 5, 2015",22.2 x 14.3 x 4.4 inches; 15.04 Ounces,JS-RANDEE-001,Jessica Simpson,No,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Baby-girls,"September 27, 2019",NaN,NaN,Josmo,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Womens,"November 19, 2018",3.27 x 2.36 x 1.14 inches; 0.02 Ounces,JWLear710b-SilvXS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,NaN,NaN,5 x 4.5 x 2.5 inches; 4.8 Ounces,NaN,NaN,NaN,NaN,LILLIWEEN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,womens,"December 21, 2012",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,Womens,"May 14, 2010",3.5 x 2 x 0.3 inches; 1.44 Ounces,SP4611-18,Amazon Collection,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
